# FurnitureGen: AI-Powered Interior Design

Jeff Barney, MSAI-495: Generative AI Image Project

# Questions:
* How do you save off your model after training?
* Do you save off the preprocessed data as well? Or is that low enough lift that you run that each session?

### Business Goal / Case Statement
Enable fast, iterative interior design by allowing users to vizualize any room in a different style. 

### Data
* I downloaded my dataset from [Kaggle](https://www.kaggle.com/datasets/stepanyarullin/interior-design-styles) 
* The dataset consists of a collection of interior design images (~1,000 per style) scraped from Houzz.com
* The data set contains 14,876 images in the training set, and 3,729 in the test set
* All images are 320x320 and in the jpg file format
* Each different style is listed below with an accompanying sample image

In [14]:
import os
import pandas as pd
from IPython.display import HTML

root_dir = '../image-project-data'
test_dir = root_dir + '/test'

# Fetch the styles from our csv of labels
raw_labels = pd.read_csv(root_dir + '/test_labels.csv')
styles = raw_labels['style'].dropna().unique()
styles_sorted = sorted(styles)

# Build a list of dicts with a style and image sample
rows = []
for style in styles:
    img_dir_path = os.path.join(test_dir, style)
    img_file_name = next((img.name for img in os.scandir(img_dir_path) if img.is_file()), None)

    img_tag = f'<img src="{os.path.join(img_dir_path, img_file_name)}" width="320">'
    rows.append({
        'Label': style,
        'Image': img_tag
    })

df = pd.DataFrame(rows)
HTML(df.to_html(escape=False))

,Label,Image
0,transitional,
1,industrial,
2,shabby-chic-style,
3,asian,
4,victorian,
5,coastal,
6,southwestern,
7,craftsman,
8,contemporary,
9,scandinavian,


preload all the data so that the cpu isn't the bottleneck

In [11]:
import tensorflow as tf
from tensorflow import keras

print(tf.config.list_physical_devices('GPU'))

[]


### Step 1 - Load and Normalize the Data

In [9]:
import os
from pathlib import Path
from PIL import Image
import numpy as np

def preprocess(imgs: np.ndarray) -> np.ndarray:
   """
   Normalize 

   Args:
      imgs (numpy.ndarray): Images to preprocess.
   Returns:
      imgs (numpy.ndarray): Preprocessed images.
   """
   imgs = imgs.astype("float32") / 255.0
   imgs = np.expand_dims(imgs, -1)
   return imgs


def load_images_to_array(is_training=False):
    root_dir = '../image-project-data/' + ('train' if is_training else 'test')
    imgs = []
    for path in Path(root_dir).rglob('*'):
        if path.suffix.lower() == '.jpg':
            with Image.open(path) as img:
                imgs.append(np.array(img))
    if not imgs:
        return np.empty((0,))  # no images found
    return np.stack(imgs, axis=0)

data = load_images_to_array()
data = preprocess(data)
print (data.shape)

(3729, 360, 360, 3, 1)


### Step 2 - Build the Encoder

In [14]:
from tensorflow.keras import backend, layers, models

EMBEDDING_DIM = 2
IMAGE_SIZE = 320

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# You might want to add more layers and then some other things between the layers?
encoder_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1), name="encoder_input")
x = layers.Conv2D(32, (3, 3), strides=2, activation="relu", padding="same")(encoder_input)
x = layers.Conv2D(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(128, (3, 3), strides=2, activation="relu", padding="same")(x)
shape_before_flattening = backend.int_shape(x)[1:]
x = layers.Flatten()(x)
z_mean = layers.Dense(EMBEDDING_DIM, name="z_mean")(x)
z_log_var = layers.Dense(EMBEDDING_DIM, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")

### Step 3 - Build the Decoder

In [15]:
decoder_input = layers.Input(shape=(EMBEDDING_DIM,), name="decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
# You might want to add more layers and then some other things between the layers?
x = layers.Conv2DTranspose(128, (3, 3), strides=2, activation="relu", padding="same")(x) 
x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
decoder_output = layers.Conv2D( 1, (3, 3), strides=1, activation="sigmoid", padding="same", name="decoder_output")(x)
decoder = models.Model(decoder_input, decoder_output)

### Step 4 - Combine the Encoder and Decoder into one VAE model

In [16]:
from tensorflow.keras import metrics, optimizers

class VAE(models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [ self.total_loss_tracker, self.reconstruction_loss_tracker, self.kl_loss_tracker, ]

    def call(self, inputs):
        """Call the model on a particular input."""
        z_mean, z_log_var, z = encoder(inputs)
        reconstruction = decoder(z)
        return z_mean, z_log_var, reconstruction

    def train_step(self, data):
        """Step run during training."""

        with tf.GradientTape() as tape:
            z_mean, z_log_var, reconstruction = self(data)
            reconstruction_loss = tf.reduce_mean( BETA * losses.binary_crossentropy(data, reconstruction, axis=(1, 2, 3)))
            kl_loss = tf.reduce_mean(tf.reduce_sum(-0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)), axis=1))
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        """Step run during validation."""
        if isinstance(data, tuple):
            data = data[0]

        z_mean, z_log_var, reconstruction = self(data)
        reconstruction_loss = tf.reduce_mean(BETA * losses.binary_crossentropy(data, reconstruction, axis=(1, 2, 3)))
        kl_loss = tf.reduce_mean(tf.reduce_sum(-0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)), axis=1))
        total_loss = reconstruction_loss + kl_loss

        return { "loss": total_loss, "reconstruction_loss": reconstruction_loss, "kl_loss": kl_loss, }

### Step 5 - Create an instance of the VAE model and train it

In [19]:
EPOCHS=1
BATCH_SIZE=10

vae = VAE(encoder, decoder)
optimizer = optimizers.Adam(learning_rate=0.0005)
vae.compile(optimizer=optimizer)
history = vae.fit(data, data, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(data, data))

### Step 6 - Visualize the latent space

### Step 7 - Test the model for data loss

In [ ]:
# Sample some images and pass them through the full flow to visualize the data loss

### Step 8 - Calculate vectors for each interior design style

In [ ]:
# Compute a latent vector for all images in our data set

# Compute a latent vector for each style
style_vector=mean(latent_vectors_style)−mean(all_latent_vectors)

### Step 9 - Visualize rooms with a different style

In [ ]:
# Create a table for each style that shows a before and after of a transformation to some other style
new_vector =old_vector + alpha*feature_vector